
# Column Transformer with Mixed Types


This example illustrates how to apply different preprocessing and feature
extraction pipelines to different subsets of features, using
`ColumnTransformer`. This is particularly handy for the
case of datasets that contain heterogeneous data types, since we may want to
scale the numeric features and one-hot encode the categorical ones.

In this example, the numeric data is standard-scaled after mean-imputation,
while the categorical data is one-hot encoded after imputing missing values
with a new category (``'missing'``).

In addition, we show two different ways to dispatch the columns to the
particular pre-processor: by column names and by column data types.

Finally, the preprocessing pipeline is integrated in a full prediction pipeline
using `Pipeline`, together with a simple classification
model.


In [8]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV


#### Step 1: Read the data set `Titanic.csv`
Print the column names. Drop 'boat','body','cabin','home.dest','name','ticket','sibsp','parch'

In [9]:
df = pd.read_csv('titanic.csv')

In [10]:
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.3375,B5,S,2,?,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,C22 C26,S,11,?,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30,1,2,113781,151.55,C22 C26,S,?,135,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"


In [11]:
df.columns

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')

In [12]:
df.drop(['boat','body','cabin','home.dest','name','ticket','sibsp','parch'], axis=1, inplace=True)  

#### Step 2: Calculate duplicates
- Report the size of the dataset
- Report if there are any duplicates using `duplicated()` method of pandas dataframe
- List all duplicate rows
- Drop all duplicate rows using `dop_duplicates` method of pandas dataframe
- Report the new shape of the dataset

In [13]:
df.shape

(1309, 6)

In [14]:
df.duplicated().sum()

np.int64(215)

In [15]:
df.drop_duplicates(inplace=True)

In [16]:
df.shape

(1094, 6)

#### Step 3: Explore the dataset for missing values
- Assign `survived' as target variable `y` and the rest as `X`
- Print info about `X` to see if there are any null values and the type of the features
- Check if all columns have only alpha numeric characters. To do that you need to use `isalnum()` method of Python which works on strings. Before using this method. we need to change all features into string using `astype('str')`
- Replace all `?` characters with `np.nan`
- Now, turn 'age','fare' into numeric again using `apply(pd.to_numeric) `
- Print the first five samples of `X`
- Summarize the number of unique values in each column using `nunique()`

In [17]:
X = df.drop('survived', axis=1)
y= df['survived']

In [18]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1094 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   pclass    1094 non-null   int64 
 1   sex       1094 non-null   object
 2   age       1094 non-null   object
 3   fare      1094 non-null   object
 4   embarked  1094 non-null   object
dtypes: int64(1), object(4)
memory usage: 51.3+ KB


In [19]:
X.head()

,pclass,sex,age,fare,embarked
0,1,female,29,211.3375,S
1,1,male,0.9167,151.55,S
2,1,female,2,151.55,S
3,1,male,30,151.55,S
4,1,female,25,151.55,S


In [20]:
cols = X.columns

In [21]:
cols

Index(['pclass', 'sex', 'age', 'fare', 'embarked'], dtype='object')

In [22]:
index = pd.Index(['pclass', 'gender', 'age', 'fare', 'embarked'], dtype='object')

In [23]:
X[cols]=X[cols].astype(str)

In [24]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1094 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   pclass    1094 non-null   object
 1   sex       1094 non-null   object
 2   age       1094 non-null   object
 3   fare      1094 non-null   object
 4   embarked  1094 non-null   object
dtypes: object(5)
memory usage: 51.3+ KB


In [25]:
X.iloc[:,2]

0           29
1       0.9167
2            2
3           30
4           25
         ...  
1301      45.5
1304      14.5
1306      26.5
1307        27
1308        29
Name: age, Length: 1094, dtype: object

In [26]:
X.iloc[:,2].str.isalnum()

0        True
1       False
2        True
3        True
4        True
        ...  
1301    False
1304    False
1306    False
1307     True
1308     True
Name: age, Length: 1094, dtype: bool

In [27]:
~X.iloc[:,2].str.isalnum()

0       False
1        True
2       False
3       False
4       False
        ...  
1301     True
1304     True
1306     True
1307    False
1308    False
Name: age, Length: 1094, dtype: bool

In [28]:
X.loc[~X.iloc[:,2].str.isalnum(),'age']

1       0.9167
15           ?
37           ?
40           ?
46           ?
         ...  
1291         ?
1294      28.5
1301      45.5
1304      14.5
1306      26.5
Name: age, Length: 172, dtype: object

In [29]:
X[~X.iloc[:,2].str.isalnum()]

,pclass,sex,age,fare,embarked
1,1,male,0.9167,151.55,S
15,1,male,?,25.925,S
37,1,male,?,26.55,S
40,1,male,?,39.6,C
46,1,male,?,31,S
...,...,...,...,...,...
1291,3,male,?,8.7125,S
1294,3,male,28.5,16.1,S
1301,3,male,45.5,7.225,C
1304,3,female,14.5,14.4542,C


In [30]:
X[~X.iloc[:,2].str.isalnum()].iloc[:,2]

1       0.9167
15           ?
37           ?
40           ?
46           ?
         ...  
1291         ?
1294      28.5
1301      45.5
1304      14.5
1306      26.5
Name: age, Length: 172, dtype: object

In [31]:
X[~X.iloc[:,3].str.isalnum()].iloc[:,3]

0       211.3375
1         151.55
2         151.55
3         151.55
4         151.55
          ...   
1301       7.225
1304     14.4542
1306       7.225
1307       7.225
1308       7.875
Name: fare, Length: 898, dtype: object

In [32]:
X = X.replace('?', np.nan)

In [33]:
type(np.nan)

float

In [34]:
num_par = {'age': 'float64', 'fare': 'float64'}

In [35]:
X[['age','fare']] = X[['age','fare']].apply(num_par)
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1094 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1094 non-null   object 
 1   sex       1094 non-null   object 
 2   age       966 non-null    float64
 3   fare      1093 non-null   float64
 4   embarked  1092 non-null   object 
dtypes: float64(2), object(3)
memory usage: 51.3+ KB


C:\Users\cleon\AppData\Local\Temp\ipykernel_33344\1198938700.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  X[['age','fare']] = X[['age','fare']].apply(num_par)


In [36]:
X.head()

,pclass,sex,age,fare,embarked
0,1,female,29.0000,211.3375,S
1,1,male,0.9167,151.5500,S
2,1,female,2.0000,151.5500,S
3,1,male,30.0000,151.5500,S
4,1,female,25.0000,151.5500,S


In [37]:
X['embarked'].unique()

array(['S', 'C', nan, 'Q'], dtype=object)

In [38]:
X['embarked']=X['embarked'].astype(str)

In [39]:
X['embarked'].unique()

array(['S', 'C', 'nan', 'Q'], dtype=object)

In [40]:
X['sex'].unique()

array(['female', 'male'], dtype=object)

#### Step 4: Use `ColumnTransformer` by selecting column by names

##############################################################################
 
 We will train our classifier with the following features:

 Numeric Features:

 * ``age``: float;
 * ``fare``: float.

 Categorical Features:

 * ``embarked``: categories encoded as strings ``{'C', 'S', 'Q'}``;
 * ``sex``: categories encoded as strings ``{'female', 'male'}``;
 * ``pclass``: ordinal integers ``{1, 2, 3}``.

 Create the preprocessing pipelines for both numeric and categorical data.
 - For numeric features, create a pipeline with using imputer with median strategy and standard scaler
 - For categorical features, use imputer with most frequent and onehot encoder
 
 Use columntransformer to transform the features
 
 Append the preprocessor to the KNeighborsClassifier


In [41]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [42]:
numeric_features = ['age', 'fare']
categorical_features = ['embarked', 'sex', 'pclass']

In [43]:
numeric_transformer = Pipeline(steps=[('imp_num', SimpleImputer(strategy='median')), ('scaler', StandardScaler())]) 

In [44]:
categorical_transformer = Pipeline(steps=[
    ('imp_cat', SimpleImputer(strategy='most_frequent', missing_values='nan')),  
    ('ohe3', OneHotEncoder(handle_unknown='ignore'))
])


In [45]:
ct2 = ColumnTransformer(transformers=[('num_ct', numeric_transformer, numeric_features),('cat_ct', categorical_transformer, categorical_features)])

In [46]:
ct2

ColumnTransformer(transformers=[('num_ct',
                                 Pipeline(steps=[('imp_num',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['age', 'fare']),
                                ('cat_ct',
                                 Pipeline(steps=[('imp_cat',
                                                  SimpleImputer(missing_values='nan',
                                                                strategy='most_frequent')),
                                                 ('ohe3',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['embarked', 'sex', 'pclass'])])

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, random_state=42)

In [48]:
clf = Pipeline(steps=[('prep', ct2), ('knn', KNeighborsClassifier())])

In [52]:
clf.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num_ct',
                                                  Pipeline(steps=[('imp_num',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'fare']),
                                                 ('cat_ct',
                                                  Pipeline(steps=[('imp_cat',
                                                                   SimpleImputer(missing_values='nan',
                                                                                 strategy='most_frequent')),
                                                                  ('ohe3',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['embarked', 'sex',
                                                   'pclass'])])),
                ('knn', KNeighborsClassifier())])

In [55]:
clf.score(X_test, y_test)

0.7883211678832117

Using the prediction pipeline in a grid search

##############################################################################

 Grid search can also be performed on the different preprocessing steps
 defined in the `ColumnTransformer` object, together with the classifier's
 hyperparameters as part of the `Pipeline`.
 We will search for both the imputer strategy of the numeric preprocessing
 and the number of neighbor parameter of the kneighbors classifier using
 `GridSearchCV`.



In [56]:

param_grid = {
    'prep__num_ct__imp_num__strategy': ['mean', 'median'],  
    'knn__n_neighbors': [3, 5, 7, 9]  
}

In [57]:
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('prep',
                                        ColumnTransformer(transformers=[('num_ct',
                                                                         Pipeline(steps=[('imp_num',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['age',
                                                                          'fare']),
                                                                        ('cat_ct',
                                                                         Pipeline(steps=[('imp_cat',
                                                                                          SimpleImputer(missing_values='nan',
                                                                                                        strategy='most_frequent')),
                                                                                         ('ohe3',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['embarked',
                                                                          'sex',
                                                                          'pclass'])])),
                                       ('knn', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'knn__n_neighbors': [3, 5, 7, 9],
                         'prep__num_ct__imp_num__strategy': ['mean', 'median']},
             scoring='accuracy')

In [58]:
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Score: {grid_search.best_score_:.2f}")

Best Parameters: {'knn__n_neighbors': 9, 'prep__num_ct__imp_num__strategy': 'mean'}
Best Score: 0.75


In [59]:
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print(f"Test Accuracy: {test_score:.2f}")

Test Accuracy: 0.78
